In this notebook we create the test data for the application scenario. To avoid having trained samples in the test set we use the test set from the finetuned HoVer model and match it with the probes test set.

In [ ]:
import pandas as pd 

df_train_probe = pd.read_pickle('../Probes/processed_datasets_llama_layer-16/mini_fact_hover_train.pkl')
df_test = pd.read_pickle('llama_finetuned_corrections/corrections_evidence_hover_test.pkl')

In [ ]:
from itertools import chain

docs_train_probe =  list(set(chain.from_iterable(df_train_probe['docs'])))
df_test = df_test[~df_test['docs'].apply(lambda x: any(item in x for item in docs_train_probe))]

We have to get the claims since they were ignored in previous research

In [ ]:
df_claim1 = pd.read_json("datasets_hover/hover_claim_evidence_generate1.json")
df_claim2 = pd.read_json("datasets_hover/hover_claim_evidence_generate2.json")
df_claim = pd.concat([df_claim1, df_claim2], ignore_index=True)
df_claim['docs'] = df_claim['supporting_facts'].apply(lambda x: list({item[0] for item in x}))
claim_list = []

for index, row in df_test.iterrows():
    docs = row['docs']
    row = df_claim[df_claim['docs'].apply(lambda x: sorted(x) == sorted(docs))]
    claim_list.append(row['claim'].values[0])
    df_test.at[index, 'claim'] = row['claim'].values[0]

In [ ]:
df_test.reset_index(drop=True, inplace=True)
df_test = df_test[['claim', 'docs', 'ground_truth_source']]

In [ ]:
# this give the final test set for the application scenario
df_test.to_pickle('application/application_data.pkl')

Additional Tests

In [ ]:
import pandas as pd
from itertools import chain

df_train_finetuned = pd.read_pickle('train_datasets_combined/corrections_evidence_combined_train_balanced.pkl')
docs_train_finetuned = list(set(chain.from_iterable(df_train_finetuned['docs'])))
df_test = pd.read_pickle('llama_finetuned_corrections/corrections_evidence_hover_test.pkl')
docs_test = list(set(chain.from_iterable(df_test['docs'])))

gen_evidence_train = df_train_finetuned['gen_evidence'].tolist()
gen_evidence_test = df_test['gen_evidence'].tolist()

for gen_e in gen_evidence_train:
    if gen_e in gen_evidence_test:
        print(gen_e)

for gen_e in gen_evidence_test:
    if gen_e in gen_evidence_train:
        print(gen_e)

In [ ]:
for doc in docs_train_finetuned:
    if doc in docs_test:
        print(doc)

In [ ]:
for doc in docs_test:
    if doc in docs_train_finetuned:
        print(doc)

In [ ]:
for doc in docs_train_probe:
    if doc in docs_test:
        print(doc)

In [ ]:
for doc in docs_test:
    if doc in docs_train_probe:
        print(doc)